In [ ]:
!pip install pandas scikit-learn nltk

In [ ]:
import nltk
import re
import string

nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

def clean_text(text):
    text = text.lower()
    text = re.sub(f"[{string.punctuation}]", "", text)
    words = text.split()
    words = [w for w in words if w not in stop_words]
    return " ".join(words)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
import pandas as pd

data = pd.read_csv("/content/wiki_movie_plots_deduped.csv", on_bad_lines='skip', engine='python')

data['plot_clean'] = data['Plot'].fillna('').apply(clean_text)

X = data['plot_clean']
y = data['Genre']

In [ ]:
print(data.columns)


Index(['Release Year', 'Title', 'Origin/Ethnicity', 'Director', 'Cast',
       'Genre', 'Wiki Page', 'Plot', 'plot_clean'],
      dtype='object')


In [ ]:
y = data['Genre']


In [ ]:
import pandas as pd
import re, string
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

def clean_text(text):
    if pd.isna(text):
        return ""
    text = text.lower()
    text = re.sub(f"[{string.punctuation}]", "", text)
    words = text.split()
    words = [w for w in words if w not in stop_words]
    return " ".join(words)

data = pd.read_csv(
    "/content/wiki_movie_plots_deduped.csv",
    on_bad_lines='skip',
    engine='python'
)

data = data.dropna(subset=['Plot', 'Genre'])

data['plot_clean'] = data['Plot'].apply(clean_text)

X = data['plot_clean']   # input
y = data['Genre']        # output ✅


In [ ]:
print(X.head())
print(y.head())


0    bartender working saloon serving drinks custom...
1    moon painted smiling face hangs park night you...
2    film minute long composed two shots first girl...
3    lasting 61 seconds consisting two shots first ...
4    earliest known adaptation classic fairytale fi...
Name: plot_clean, dtype: object
0    unknown
1    unknown
2    unknown
3    unknown
4    unknown
Name: Genre, dtype: object


In [ ]:
data = pd.read_csv(
    "/content/wiki_movie_plots_deduped.csv",
    on_bad_lines='skip',
    engine='python'
)

data = data.dropna(subset=['Plot'])

data['plot'] = data['Plot'].apply(clean_text)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(max_features=5000)

X_tfidf = tfidf.fit_transform(X)


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_tfidf, y, test_size=0.2, random_state=42
)


In [ ]:
from sklearn.naive_bayes import MultinomialNB

nb_model = MultinomialNB()
nb_model.fit(X_train, y_train)


MultinomialNB()

In [ ]:

from sklearn.linear_model import LogisticRegression

lr_model = LogisticRegression(max_iter=1000)
lr_model.fit(X_train, y_train)


LogisticRegression(max_iter=1000)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(
    max_features=2000,
    min_df=5,
    max_df=0.8
)

X_tfidf = tfidf.fit_transform(X)


In [ ]:
from sklearn.svm import LinearSVC

svm_model = LinearSVC()
svm_model.fit(X_train, y_train)


LinearSVC()

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

y_pred = svm_model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy: 0.39022051773729627
                                   precision    recall  f1-score   support

                           action       0.05      0.06      0.06        17
                    action comedy       0.00      0.00      0.00         1
                   action, comedy       0.00      0.00      0.00         1
                  action, western       0.00      0.00      0.00         1
                            adult       0.00      0.00      0.00         3
                        adventure       0.29      0.39      0.33        70
                 adventure, drama       0.00      0.00      0.00         1
               adventure, fantasy       0.00      0.00      0.00         1
                         animated       0.27      0.30      0.29        20
                 animated musical       0.00      0.00      0.00         1
                   animated short       0.55      0.33      0.41        18
                 animated, family       0.00      0.00      0.00     

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer


tfidf = TfidfVectorizer(max_features=5000)

tfidf.fit(X)

def predict_genre(plot):
    plot = clean_text(plot)
    plot_vec = tfidf.transform([plot])
    return svm_model.predict(plot_vec)[0]

new_plot = "A brave warrior saves the kingdom from dark forces"
print("Predicted Genre:", predict_genre(new_plot))

Predicted Genre: drama
